# Automated G0W0 band interpolation via Wannierization, doing also the convergence.

The interpolation of G0W0 eigenvalues by means of Wannierization can be performed automatically within the YamboWannier90WorkChain delivered in the aiida-yambo-wannier90 plugin, as described in [Bonacci, M., Qiao, J., Spallanzani, N. et al. Towards high-throughput many-body perturbation theory: efficient algorithms and automated workflows. npj Comput Mater 9, 74 (2023)](https://doi.org/10.1038/s41524-023-01027-2).

In [1]:
from aiida import orm, load_profile
load_profile()

Profile<uuid='b35700dae723411ea16ebc82d58f16bc' name='mb'>

In [2]:
from aiida_wannier90_workflows.utils.workflows.builder.serializer import print_builder 
from aiida_wannier90_workflows.utils.workflows.builder.setter import set_parallelization, set_num_bands
from aiida_wannier90_workflows.utils.workflows.builder.submit import submit_and_add_group 
from aiida_wannier90_workflows.common.types import WannierProjectionType

from aiida_quantumespresso.workflows.pw.base import PwBaseWorkChain

from aiida_wannier90_workflows.workflows.bands import Wannier90BandsWorkChain
from aiida_wannier90_workflows.workflows.base.wannier90 import (
        Wannier90BaseWorkChain,
    )

from aiida_yambo_wannier90.workflows import YamboWannier90WorkChain

/scratch/mbonacci/mambaforge/envs/minitutorial/lib/python3.10/site-packages/aiida/engine/processes/ports.py:145: UserWarning: default of input port `settings` is a `Node` instance, which can lead to unexpected side effects. It is advised to use a lambda instead, e.g.: `default=lambda: orm.Int(5)`.
  warnings.warn(UserWarning(message))  # pylint: disable=no-member


## minimal inputs needed for protocols

In [3]:
codes = {
        "pw": "pw-6.8-p2w_ham@hydralogin",
        "pw2wannier90": "pw2wannier90-6.8-p2w_ham@hydralogin",
        "projwfc": "projwfc-7.1@hydralogin",
        "wannier90": "w90@hydralogin",
        "yambo": "yambo-5.1@hydralogin",
        "p2y": "p2y-5.1@hydralogin",
        "ypp": "ypp-5.1@hydralogin",
        "gw2wannier90": "gw2wannier90@hydralogin",
    }

In [ ]:
from aiida.orm import QueryBuilder
qb = QueryBuilder()
qb.append(orm.Group, filters={'label': 'hBN/bulk'}, tag='group')
qb.append(orm.StructureData, with_group='group')

loaded_structure_id = qb.all()[0][0].pk

In [4]:
options = {
    'pseudo_family':"PseudoDojo/0.4/PBE/SR/standard/upf",
    'protocol':'fast',
    #'parent_id':274, #not necessary to set; if you want it, take ytheour previously nscf id (pk) to skip the DFT part.
    'structure_id':loaded_structure_id,
}

In [5]:
projections="analytic"

if projections=="analytic":
    wannier_projection_type=WannierProjectionType.ANALYTIC
else:
    WannierProjectionType.ATOMIC_PROJECTORS_QE 


In [6]:
builder = YamboWannier90WorkChain.get_builder_from_protocol(
        codes=codes,
        structure=orm.load_node(options["structure_id"]),
        pseudo_family=options["pseudo_family"],
        protocol="fast",
        wannier_projection_type=wannier_projection_type,
    )

Summary of the main inputs:
BndsRnXp = 400
GbndRnge = 400
NGsBlkXp = 16 Ry
FFTGvecs = 42 Ry


kpoint mesh for nscf: [10, 10, 4]


/scratch/mbonacci/mambaforge/envs/minitutorial/lib/python3.10/site-packages/aiida_quantumespresso/calculations/pw.py:211: UserWarning: `parent_folder` not provided for `nscf` calculation. For work chains wrapping this calculation, you can disable this warning by excluding the `parent_folder` when exposing the inputs of the `PwCalculation`.
  warnings.warn(


Summary of key input parameters:
  Formula: B2N2
  PseudoFamily: PseudoDojo/0.4/PBE/SR/standard/upf
  ElectronicType: METAL
  SpinType: NONE
  WannierProjectionType: ANALYTIC
  WannierDisentanglementType: SMV
  WannierFrozenType: ENERGY_FIXED
  num_bands: 32
  num_wann: 16
  mp_grid: [15, 15, 5]

Summary of the inputs:{'arguments': ['wannier'], 'variables': {'BoseTemp': [0, 'eV'], 'WriteAMU': ''}}

Summary of the inputs:{'arguments': ['infver', 'QPDBs', 'QPDB_merge'], 'variables': {'BoseTemp': [0, 'eV']}}



/scratch/mbonacci/mambaforge/envs/minitutorial/codes/aiida-yambo/aiida_yambo/workflows/ypprestart.py:145: UserWarning: You must provide a parent folder calculation, either YPP or YAMBO
  warnings.warn('You must provide a parent folder calculation, either YPP or YAMBO')
/scratch/mbonacci/mambaforge/envs/minitutorial/lib/python3.10/site-packages/aiida_quantumespresso/calculations/pw.py:211: UserWarning: `parent_folder` not provided for `nscf` calculation. For work chains wrapping this calculation, you can disable this warning by excluding the `parent_folder` when exposing the inputs of the `PwCalculation`.
  warnings.warn(


In [46]:
# Increase ecutwfc, to have hig
params = builder.yambo.ywfl.scf.pw.parameters.get_dict()
params["SYSTEM"]["ecutwfc"] = 100
builder.yambo.ywfl.scf.pw.parameters = orm.Dict(dict=params)
params = builder.yambo.ywfl.nscf.pw.parameters.get_dict()
params["SYSTEM"]["ecutwfc"] = 100
builder.yambo.ywfl.nscf.pw.parameters = orm.Dict(dict=params)

parallelization = dict(
    max_wallclock_seconds=24 * 3600,
    # num_mpiprocs_per_machine=48,
    #npool=4,
    num_machines=1,
)
set_parallelization(
    builder["yambo"]["ywfl"]["scf"],
    parallelization=parallelization,
    process_class=PwBaseWorkChain,
)
set_parallelization(
    builder["yambo"]["ywfl"]["nscf"],
    parallelization=parallelization,
    process_class=PwBaseWorkChain,
)
set_parallelization(
    builder["yambo_qp"]["scf"],
    parallelization=parallelization,
    process_class=PwBaseWorkChain,
)
set_parallelization(
    builder["yambo_qp"]["nscf"],
    parallelization=parallelization,
    process_class=PwBaseWorkChain,
)

set_parallelization(
    builder["wannier90"],
    parallelization=parallelization,
    process_class=Wannier90BandsWorkChain,
)
set_parallelization(
    builder["wannier90_qp"],
    parallelization=parallelization,
    process_class=Wannier90BaseWorkChain,
)

builder['yambo']['parameters_space']= orm.List(list=[{'conv_thr': 10,
                                 'conv_thr_units': '%',
                                 'convergence_algorithm': 'new_algorithm_1D',
                                 'delta': 8,
                                 'max': 84,
                                 'max_iterations': 4,
                                 'start': 21,
                                 'steps': 4,
                                 'stop': 58,
                                 'var': ['FFTGvecs']},])

builder['yambo']['workflow_settings']= orm.Dict(dict= {'bands_nscf_update': 'all-at-once',
                            'skip_pre': True,
                            'type': '1D_convergence',
                            'what': ['gap_GG']},)



#### START computational resources settings.
builder['yambo']['ywfl']['yres']['yambo']['metadata'] = {'options': {'stash': {}, 'resources': {'num_machines': 1, 'num_cores_per_mpiproc': 1, 'num_mpiprocs_per_machine': 16}, 'max_wallclock_seconds': 86400,
                                            'withmpi': True, 'queue_name': 's3par', 'prepend_text': 'export OMP_NUM_THREADS=1'}}

builder['yambo']['ywfl']['scf']['pw']['metadata'] = {'options': {'stash': {}, 'resources': {'num_machines': 1, 'num_cores_per_mpiproc': 16, 'num_mpiprocs_per_machine': 1}, 'max_wallclock_seconds': 86400,
                                        'withmpi': True, 'queue_name': 's3par', 'prepend_text': 'export OMP_NUM_THREADS=16'}}
    
builder['yambo']['ywfl']['nscf']['pw']['metadata'] = {'options': {'stash': {}, 'resources': {'num_machines': 1, 'num_cores_per_mpiproc': 2, 'num_mpiprocs_per_machine':8}, 'max_wallclock_seconds': 86400,
                                        'withmpi': True, 'queue_name': 's3par', 'prepend_text': 'export OMP_NUM_THREADS=2'}}


builder['yambo_qp']['additional_parsing'] = orm.List(list=['gap_GG'])

builder['yambo_qp']['yres']['yambo']['metadata'] = {'options': {'stash': {}, 'resources': {'num_machines': 1, 'num_cores_per_mpiproc': 1, 'num_mpiprocs_per_machine': 16}, 'max_wallclock_seconds': 86400,
                                            'withmpi': True, 'queue_name': 's3par', 'prepend_text': 'export OMP_NUM_THREADS=1'}}

builder['yambo_qp']['scf']['pw']['metadata'] = {'options': {'stash': {}, 'resources': {'num_machines': 1, 'num_cores_per_mpiproc': 16, 'num_mpiprocs_per_machine': 1}, 'max_wallclock_seconds': 86400,
                                        'withmpi': True, 'queue_name': 's3par', 'prepend_text': 'export OMP_NUM_THREADS=16'}}
    
builder['yambo_qp']['nscf']['pw']['metadata'] = {'options': {'stash': {}, 'resources': {'num_machines': 1, 'num_cores_per_mpiproc': 1, 'num_mpiprocs_per_machine': 16}, 'max_wallclock_seconds': 86400,
                                        'withmpi': True, 'queue_name': 's3par', 'prepend_text': 'export OMP_NUM_THREADS=1'}}


builder['wannier90_qp']['wannier90']['metadata'] = {'options': {'stash': {}, 'resources': {'num_machines': 1, 'num_cores_per_mpiproc': 16, 'num_mpiprocs_per_machine': 1}, 'max_wallclock_seconds': 86400,
                                        'withmpi': True, 'queue_name': 's3par', 'prepend_text': 'export OMP_NUM_THREADS=16'}}



builder['wannier90']['nscf']['pw']['metadata'] = {'options': {'stash': {}, 'resources': {'num_machines':1, 'num_cores_per_mpiproc': 1, 'num_mpiprocs_per_machine': 16}, 'max_wallclock_seconds': 86400,
                                        'withmpi': True, 'queue_name': 's3par', 'prepend_text': 'export OMP_NUM_THREADS=1'}}


builder['wannier90']['scf']['pw']['metadata'] = {'options': {'stash': {}, 'resources': {'num_machines': 1, 'num_cores_per_mpiproc': 16, 'num_mpiprocs_per_machine': 1}, 'max_wallclock_seconds': 86400,
                                        'withmpi': True, 'queue_name': 's3par', 'prepend_text': 'export OMP_NUM_THREADS=16'}}

builder['wannier90']['pw2wannier90']['pw2wannier90']['metadata'] = {'options': {'stash': {}, 'resources': {'num_machines': 1, 'num_cores_per_mpiproc': 8, 'num_mpiprocs_per_machine':2}, 'max_wallclock_seconds': 86400,
                                        'withmpi': True, 'queue_name': 's3par', 'prepend_text': 'export OMP_NUM_THREADS=8'}}

builder['wannier90']['wannier90']['wannier90']['metadata'] = {'options': {'stash': {}, 'resources': {'num_machines': 1, 'num_cores_per_mpiproc':2, 'num_mpiprocs_per_machine': 8}, 'max_wallclock_seconds': 86400,
                                        'withmpi': True, 'queue_name': 's3par', 'prepend_text': 'export OMP_NUM_THREADS=2'}}

preprend_ypp_w = builder['ypp']['ypp']['metadata']['options']['prepend_text']
builder['ypp']['ypp']['metadata'] = {'options': {'stash': {}, 'resources': {'num_machines': 1, 'num_cores_per_mpiproc': 16, 'num_mpiprocs_per_machine': 1}, 'max_wallclock_seconds': 86400,
                                        'withmpi': True, 'queue_name': 's3par', 'prepend_text': preprend_ypp_w}}

preprend_ypp_qp = builder['ypp_QP']['ypp']['metadata']['options']['prepend_text']
builder['ypp_QP']['ypp']['metadata'] = {'options': {'stash': {}, 'resources': {'num_machines': 1, 'num_cores_per_mpiproc': 16, 'num_mpiprocs_per_machine': 1}, 'max_wallclock_seconds': 86400,
                                        'withmpi': True, 'queue_name': 's3par', 'prepend_text': preprend_ypp_qp}}

builder['gw2wannier90']['metadata'] = {'options': {'stash': {}, 'resources': {'num_machines':1, 'num_cores_per_mpiproc': 1, 'num_mpiprocs_per_machine': 16}, 'max_wallclock_seconds': 86400,
                                        'queue_name': 's3par', 'prepend_text': 'export OMP_NUM_THREADS=1'}}

#### END computational resources settings.

In [47]:
# SKIP Convergence:
# 1. pop "yambo"
# 2. pop the "parent_folder" of "yambo_qp"
# 3. add the "GW_mesh"  KpointsData input.

#builder.pop('yambo') # to skip the convergence
#builder['yambo_qp'].pop('parent_folder',None) # to skip the convergence

In [48]:
# SKIP the yambo QP step:
# this will skip yambo_qp, but be sure to provide QP_DB and parent_folder to ypp inputs.
# In general, you can do this if you have already the yambo results.
#builder.pop('yambo_qp')
#builder.ypp.ypp.QP_DB = orm.load_node(12501)
#builder.ypp.parent_folder = orm.load_node(12305).outputs.remote_folder

In [49]:
# SET custom K-MESH:
kpoints = orm.KpointsData() # to skip the convergence
kpoints.set_kpoints_mesh([8,8,4]) # to skip the convergence
builder.GW_mesh = kpoints # to skip the convergence

In [50]:
#If we want W90 to use the GW mesh a priori, set the following to True. Usually, if converged for GW, it should be ok also for the Wannierization.
builder.kpoints_force_gw = orm.Bool(True)

In [51]:
# START projections settings:
    
set_num_bands(
    builder=builder.wannier90, 
    num_bands=32,                   # KS states used in the Wannierization
    #exclude_bands=range(1,5), 
    process_class=Wannier90BandsWorkChain)

params = builder.wannier90.wannier90.wannier90.parameters.get_dict()

## START explicit atomic projections:
if projections=="analytic":

    del builder.wannier90.projwfc
    builder.wannier90.wannier90.auto_energy_windows = False
    builder.wannier90.wannier90.shift_energy_windows = True
    params['num_wann'] = 16
    builder.wannier90.wannier90.wannier90.projections = orm.List(list=['B:s', 'B:p', 'N:s', 'N:p'])
    builder.wannier90_qp.wannier90.projections = builder.wannier90.wannier90.wannier90.projections
    params.pop('auto_projections', None) # Uncomment this if you want analytic atomic projections

    #
    # The following line can be also deleted.
    builder['wannier90']['pw2wannier90']['pw2wannier90']['parameters'] = orm.Dict(dict={'inputpp': {'atom_proj': False}})

## END explicit atomic projections:

# optional settings.
#params.pop('dis_proj_min', None)
#params.pop('dis_proj_max', None)
#params['num_wann'] = 16
#params['dis_froz_max'] = 2

params = orm.Dict(dict=params)
builder.wannier90.wannier90.wannier90.parameters = params
builder.wannier90_qp.wannier90.parameters = params

# END projections settings.

In [52]:
# START QP settings:
#builder['yambo_qp']['parent_folder'] = orm.load_node(13004).outputs.remote_folder
if "yambo_qp" in builder.keys():
    builder['yambo_qp']['QP_subset_dict'] = orm.Dict(dict={
                                        'qp_per_subset':50,
                                        'parallel_runs':4,
                                })

    builder['yambo_qp']['yres']['yambo']['parameters'] = orm.Dict(dict={'arguments': ['dipoles', 'ppa', 'HF_and_locXC', 'gw0', 'rim_cut'],
        'variables': {'Chimod': 'hartree',
        'DysSolver': 'n',
        'GTermKind': 'BG',
        'PAR_def_mode': 'workload',
        'X_and_IO_nCPU_LinAlg_INV': [1, ''],
        'RandQpts': [5000000, ''],
        'RandGvec': [100, 'RL'],
        'NGsBlkXp': [6, 'Ry'],
        'FFTGvecs': [40, 'Ry'],
        'BndsRnXp': [[1, 150], ''],
        'GbndRnge': [[1, 150], ''],
        'QPkrange': [[[1, 1, 32, 32]], '']}})
    
if "yambo" in builder.keys():

    builder['yambo']["ywfl"]['yres']['yambo']['parameters'] = orm.Dict(dict={'arguments': ['dipoles', 'ppa', 'HF_and_locXC', 'gw0', 'rim_cut'],
        'variables': {'Chimod': 'hartree',
        'DysSolver': 'n',
        'GTermKind': 'BG',
        'PAR_def_mode': 'workload',
        'X_and_IO_nCPU_LinAlg_INV': [1, ''],
        'RandQpts': [5000000, ''],
        'RandGvec': [100, 'RL'],
        'NGsBlkXp': [6, 'Ry'],
        'FFTGvecs': [40, 'Ry'],
        'BndsRnXp': [[1, 150], ''],
        'GbndRnge': [[1, 150], ''],
        'QPkrange': [[[1, 1, 32, 32]], '']}})

# END QP settings.

In [7]:
print_builder(builder)

{'clean_workdir': False,
 'gw2wannier90': {'code': 'gw2wannier90@hydralogin<3188>'},
 'structure': 'B2N2<161>',
 'wannier90': {'nscf': {'kpoints': '1125 kpts<None>',
                        'kpoints_force_parity': False,
                        'pw': {'code': 'pw-6.8-p2w_ham@hydralogin<2939>',
                               'metadata': {'options': {'max_wallclock_seconds': 43200,
                                                        'resources': {'num_machines': 1},
                                                        'withmpi': True}},
                               'parameters': {'CONTROL': {'calculation': 'nscf',
                                                          'etot_conv_thr': 4e-05,
                                                          'forc_conv_thr': 0.0001,
                                                          'restart_mode': 'from_scratch',
                                                          'tprnfor': True,
                                         

### Computational resources

In [54]:
from aiida.engine import submit

In [55]:
run = None

In [56]:
if run:
    print('run is already running -> {}'.format(run.pk))
    print('sure that you want to run again?, if so, copy the else instruction in the cell below and run!')
else:
    run = submit(builder)

print(run)

uuid: 5d2e68ef-bb0a-48ff-80cc-dc3749d60b1f (pk: 16392) (aiida.workflows:yambo_wannier90)


In [59]:
!verdi process report {run.pk}

2024-01-25 18:35:43 [8058 | REPORT]: [16392|YamboWannier90WorkChain|run_seekpath]: launching seekpath: B2N2 -> B2N2
2024-01-25 18:35:45 [8059 | REPORT]: [16392|YamboWannier90WorkChain|run_yambo_convergence]: launching YamboConvergence<16398>
2024-01-25 18:35:47 [8060 | REPORT]:   [16398|YamboConvergence|start_workflow]: Workflow type: 1D_convergence; looking for convergence of ['gap_GG']
2024-01-25 18:35:47 [8061 | REPORT]:   [16398|YamboConvergence|start_workflow]: Workflow initilization step completed, the parameters will be: ['FFTGvecs'].
2024-01-25 18:35:47 [8062 | REPORT]:   [16398|YamboConvergence|has_to_continue]: Still iteration on ['FFTGvecs']
2024-01-25 18:35:47 [8063 | REPORT]:   [16398|YamboConvergence|pre_needed]: skipping pre, debug mode
2024-01-25 18:35:48 [8064 | REPORT]:   [16398|YamboConvergence|next_step]: New parameters are: {'FFTGvecs': 21}
2024-01-25 18:35:50 [8065 | REPORT]:   [16398|YamboConvergence|next_step]: New parameters are: {'FFTGvecs': 37}
2024-01-25 18: